# HW6
## Umair Hussain
### hussa906



In [8]:
import pymc as pm
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Load the CSV file into a DataFrame
df = pd.read_csv("data.csv", index_col=0)

# Print the DataFrame to inspect its structure
print(df.head())

# Select only the 'Age' column for X
X = df[['Age']]

# Check the structure of X
print(X.head())

# Select the 'Approved' column for y
y = df[['Approved']]

print(y.head())

      Age   Debt  Married  BankCustomer     Industry Ethnicity  YearsEmployed  \
ID                                                                              
1   30.83  0.000        1             1  Industrials     White           1.25   
2   58.67  4.460        1             1    Materials     Black           3.04   
3   24.50  0.500        1             1    Materials     Black           1.50   
4   27.83  1.540        1             1  Industrials     White           3.75   
5   20.17  5.625        1             1  Industrials     White           1.71   

    PriorDefault  Employed  CreditScore  DriversLicense       Citizen  \
ID                                                                      
1              1         1            1               0       ByBirth   
2              1         1            6               0       ByBirth   
3              1         0            0               0       ByBirth   
4              1         1            5               1       ByBir

In [9]:
with pm.Model() as model:
    # Standard normal priors for unknown beta parameters
    b0 = pm.Normal('b0', mu=0, sigma=1)
    b1 = pm.Normal('b1', mu=0, sigma=1)
    mu = b0 + b1*X
    Y_obs = pm.Bernoulli('Y_obs',logit_p=mu, observed=y)

with model:
    idata = pm.sample()

## Part 2

Normal Priors for Betas

$$\text{posterior} \propto \Pi_{j=1}^N\frac{1}{\sqrt{2\pi}} e^{\frac{-1}{2}(\frac{B_j}{s_j})^2} \cdot \Pi_{i=1}^N\frac{1}{\sqrt{2\pi}}e^{\frac{-1}{2}({y_i - x_i^TB})^2}$$
$$\propto \Pi_{j=1}^Ne^{\frac{-1}{2}(\frac{B_j}{s_j})^2} \cdot \Pi_{i=1}^Ne^{\frac{-1}{2}({y_i - x_i^TB})^2}$$

$$\text{Log Posterior} \propto log(\Pi_{j=1}^N e^{\frac{-1}{2}(\frac{B_j}{s_j})^2} \cdot \Pi_{i=1}^Ne^{\frac{-1}{2}({y_i - x_i^TB})^2})$$
$$\propto log(\Pi_{j=1}^N e^{\frac{-1}{2}(\frac{B_j}{s_j})^2}) + log(\Pi_{i=1}^Ne^{\frac{-1}{2}({y_i - x_i^TB})^2})$$

$$\propto \sum_{j=1}^N\frac{-1}{2}(\frac{B_j}{s_j})^2 - \sum_{i=1}^N\frac{1}{2}({y_i - x_i^TB})^2$$
$$\propto \frac{1}{2}\sum_{i=1}^N({y_i - x_i^TB})^2 + \frac{1}{2}\sum_{j=1}^N(\frac{1}{s_j})^2B_j^2$$

This looks like the ridge penalty form in which the terms in $\frac{1}{s_j}^2$ are simply $\lambda$

Laplace Priors for Betas

$$\text{posterior} \propto \Pi_{j=1}^N\frac{1}{2s_j} e^{-\frac{|B_j|}{s_j}} \cdot \Pi_{i=1}^N\frac{1}{\sqrt{2\pi}}e^{\frac{-1}{2}({y_i - x_i^TB})^2}$$
$$\propto \Pi_{j=1}^Ne^{-\frac{|B_j|}{s_j}} \cdot \Pi_{i=1}^Ne^{\frac{-1}{2}({y_i - x_i^TB})^2}$$
$$\text{Log Posterior} \propto log(\Pi_{j=1}^N e^{-\frac{|B_j|}{s_j}} \cdot \Pi_{i=1}^N\frac{1}{\sqrt{2\pi}}e^{\frac{-1}{2}({y_i - x_i^TB})^2})$$
$$\propto log(\Pi_{j=1}^N e^{-\frac{|B_j|}{s_j}}) + log(\Pi_{i=1}^N\frac{1}{\sqrt{2\pi}}e^{\frac{-1}{2}({y_i - x_i^TB})^2})$$
$$\propto \sum_{j=1}^N-\frac{|B_j|}{s_j} - \sum_{i=1}^N\frac{1}{2}({y_i - x_i^TB})^2$$
$$\propto \frac{1}{2}\sum_{i=1}^N({y_i - x_i^TB})^2 + \sum_{j=1}^N\frac{|B_j|}{s_j}$$

Looks like lasso penalty form in which the terms $\frac{1}{s_j}$ are simply $\lambda$

Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior.